In [451]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import urllib.request
import copy
import pandas as pd
from newspaper import Article
import re
import nltk
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import string
import pandas as pd
import re 
# словарь для отнесения к теме
data = pd.read_csv('vocab.csv')
stopwords_ru=list(data['0'])
# символы для удаления
patterns = "[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
morph = MorphAnalyzer()

In [452]:
# новостей про ИБ с кждого сайта 
it_news_count=10
# новостей не про ИБ с каждого сайта
simpl_news_count=10
# фэйков генерить
fake_news=40

In [453]:
# лемматизация
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
#     print("len=",len(doc.split()))
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    raznost=len(doc.split())-len(tokens)
#     print("/////////////////////////////////////")
#     print(raznost)
#     print(round(0.2*len(doc.split())))
#     print()
    if  raznost >= round(0.07*len(doc.split())):
        return 0
    else:
        return 112

In [454]:
# получить текст новостной статьи и проверяем его на принадлежность теме
def get_news(url, data_,flag,datc):
    news = Article(url, language='ru')
    news.download()
    news.parse()
    data_frame=pd.DataFrame()
    el=str(news.text)
    if flag==1:
        text = copy.deepcopy(el[el.find("\n",el.find("\n")+2)+2:].lower())
    else:
        text=copy.deepcopy(el)
    data_frame=pd.DataFrame()
    data_frame=pd.concat([data_frame,pd.DataFrame([text])])
    flag=data_frame[0].apply(lemmatize)
    if (datc==0 or datc==1) and flag[0]==0:
        data_=pd.concat([data_,pd.DataFrame({'News':[copy.deepcopy(el)], 'InfSec':[1],'FAKE?':[0],'Link':[copy.deepcopy(url)]})])
        return data_,0
    elif (datc==0 or datc==2) and flag[0]!=0:
        data_=pd.concat([data_,pd.DataFrame({'News':[copy.deepcopy(el)], 'InfSec':[0],'FAKE?':[0],'Link':[copy.deepcopy(url)]})])
        return data_,112
    return None,None
# 0 - текст новости
# 1 - относится ли к ИБ (1-да, 0- нет)
# 2 - является фейком (1-да,0 - нет)
# 3 - ссылка на новость

In [455]:
def flag_func(flag,it_news,simpl_news,datchik):
    #     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    global it_news_count
    global simpl_news_count
    if flag==0:
        it_news=it_news+1
    else:
        simpl_news=simpl_news+1
    if datchik==0:
        if it_news>=it_news_count:
            datchik=2
        if simpl_news>=simpl_news_count:
            datchik=1
    return it_news,simpl_news,datchik

In [456]:
# выясняем сколько страниц на сайие
def secure_lab_page():
    url="https://www.securitylab.ru/news/page1_1.php"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
    box = soup.find('div', class_='page-picker')
    st = str(box.find('input'))
    num_page=''
    for i in range(12,len(st)):
        if st[i]=='"':
            break
        else:
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [457]:
# получаем новости
def secure_lab_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(1,num_page+1):
#     for k in range(6,8):
        url="https://www.securitylab.ru/news/page1_"+str(k)+".php"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
        box = soup.find_all('a', class_='article-card inline-card')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
#                 if k==7:
#                     print(box[i])
                urls=''
                for j in range (42,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
#                         print("https://www.securitylab.ru"+urls)
#                         print(datchik)
                        temp_dataframe,flag=get_news("https://www.securitylab.ru"+urls,dataframe,1,datchik)
#                         print(flag)
                        if flag!=None:
#                             print(urls)
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except ValueError:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [458]:
# выясняем сколько страниц на сайие
def xakep_page():
    url="https://xakep.ru/category/news/"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
    box = soup.find('span', class_='pages')
    st=str(box)
    num_page=''
    for i in range(34,len(st)):
        if st[i]=='<':
            break
        elif re.search(r'[0-9]', st[i]):
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [459]:
def xakep_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(1,num_page+1):
        url="https://xakep.ru/category/news/page/"+str(k)+"/"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='bdaia-blocks-container'))
        box = soup.find_all('h3',class_='entry-title')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
                urls=''
                for j in range (33,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
#                         dataframe=get_news(urls,dataframe,2)
                        temp_dataframe,flag=get_news(urls,dataframe,2,datchik)
                        if flag!=None:
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except ValueError:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [460]:
# выясняем сколько страниц на сайие
def anti_malware_page():
    url="https://www.anti-malware.ru/news?page=0"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('li',class_='pager-last'))
    box = soup.find('a')
    st=str(box)
    num_page=''
    for i in range(20,len(st)):
        if st[i]=='"':
            break
        elif re.search(r'[0-9]', st[i]):
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [461]:
def anti_malware_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(0,num_page):
        url="https://www.anti-malware.ru/news?page="+str(k)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='region region-content'))
        box = soup.find_all('h2')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
                urls=''
                for j in range (13,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
#                         dataframe=get_news('https://www.anti-malware.ru'+urls,dataframe,2)
                        temp_dataframe,flag=get_news('https://www.anti-malware.ru'+urls,dataframe,2,datchik)
                        if flag!=None:
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except ValueError:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [462]:
# выясняем сколько страниц на сайие
def securitymedia_page():
    url="https://securitymedia.org/news/?PAGEN_2=1"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='modern-page-navigation'))
    box = soup.find_all('a')
    st=str(box[len(box)-2])
    num_page=''
    for i in range(24,len(st)):
        if st[i]=='"':
            break
        else:
            num_page=num_page+st[i]
#     print(num_page)
    num_page=int(num_page)
    return num_page

In [463]:
def securitymedia_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(0,num_page):
        url="https://securitymedia.org/news/?PAGEN_2="+str(k)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='col-md-9'))
        box = soup.find_all('a',class_="text-dark")
        link=[]
        for i in range (0, len(box),2):
            try:
                stroka_url=str(box[i])
                urls=''
                
                for j in range (27,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
#                         dataframe=get_news('https://securitymedia.org'+urls,dataframe,2)
                        temp_dataframe,flag=get_news('https://securitymedia.org'+urls,dataframe,2,datchik)
                        if flag!=None:
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except ValueError:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [464]:
result_dataframe=pd.DataFrame()

In [465]:
page=securitymedia_page()
site_pd=securitymedia_news(copy.deepcopy(page))
result_dataframe=pd.concat([site_pd,result_dataframe])

len dataframe= 16  it_news= 6  simpl_news= 10   странициа сайта= 0
len dataframe= 20  it_news= 10  simpl_news= 10   странициа сайта= 1


In [466]:
page=anti_malware_page()
# тут страницы с нулевой
site_pd=anti_malware_news(copy.deepcopy(page+1))
result_dataframe=pd.concat([site_pd,result_dataframe])

len dataframe= 10  it_news= 6  simpl_news= 4   странициа сайта= 0
len dataframe= 17  it_news= 7  simpl_news= 10   странициа сайта= 1
len dataframe= 20  it_news= 10  simpl_news= 10   странициа сайта= 2


In [467]:
page=xakep_page()
site_pd=xakep_news(copy.deepcopy(page))
result_dataframe=pd.concat([site_pd,result_dataframe])

len dataframe= 20  it_news= 10  simpl_news= 10   странициа сайта= 1


In [468]:
page=secure_lab_page()
site_pd=secure_lab_news(copy.deepcopy(page))
result_dataframe=pd.concat([site_pd,result_dataframe])

len dataframe= 15  it_news= 8  simpl_news= 7   странициа сайта= 1
len dataframe= 20  it_news= 10  simpl_news= 10   странициа сайта= 2


,News,InfSec,FAKE?,Link
0,За первое полугодие 2022 года из России уедут ...,0,0,https://www.securitylab.ru/news/531931.php
0,Криптосообщество обмануло хакера и вернуло укр...,0,0,https://www.securitylab.ru/news/531930.php
0,Суд поддержал законность штрафа Apple на 12 мл...,1,0,https://www.securitylab.ru/news/531929.php
0,ESET: Буткит ESPecter все еще угрожает UEFI\n\...,1,0,https://www.securitylab.ru/news/531928.php
0,85% покинувших Россию IT-специалистов вернулис...,0,0,https://www.securitylab.ru/news/531927.php
...,...,...,...,...
0,Специалисты Google рекомендуют пользователям Z...,1,0,https://securitymedia.org/news/novaya-uyazvimo...
0,Human Rights Watch (HRW) изучила более 150 про...,1,0,https://securitymedia.org/news/sotni-prilozhen...
0,"Новый веб-сайт, на котором были опубликованы п...",1,0,https://securitymedia.org/news/google-russkie-...
0,Председатель временной комиссии Совфеда по инф...,1,0,https://securitymedia.org/news/sovfed-rossiyu-...
